In [1]:
import pandas as pd
import os
import wiggum as wg
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the LabeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = wg.LabeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['independent','groupby'], 'year':'independent', 'driver_gender':['trend','groupby'], 
         'driver_race':['independent','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'dependent', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'dependent', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'dependent', 'num_stops':'dependent'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical','decriminalization':'categorical', 
             'medical':'categorical', 'recreational':'categorical', 'no_reforms':False,}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops',
             'contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:365: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v
/home/smb/anaconda3/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:341: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[independent, groupby]",False,NaN
year,int64,ordinal,independent,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[independent, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

True

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = wg.LabeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[independent, splitby]",False,NaN
year,int64,ordinal,[independent],False,NaN
driver_gender,object,categorical,"[[independent, dependent], splitby]",False,NaN
driver_race,object,categorical,"[independent, splitby]",False,NaN
decriminalization,int64,categorical,[splitby],False,NaN
medical,int64,categorical,[splitby],False,NaN
recreational,int64,categorical,[splitby],False,NaN
no_reforms,int64,False,[splitby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_intersectional(['driver_gender','driver_race'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0,F_White
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0,M_White
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian


We can also pass more variables and lengths of the tuples that we want it to combine.  For example if we provide 3 categorical variables we can do both pairs and triples of the variables.

In [11]:
labeled_df.add_intersectional(['driver_gender','driver_race','state'],[2,3])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.055556,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ


In [12]:
labeled_df.add_all_dpgmm(qual_thresh =.2)
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.055556,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ


In [13]:
labeled_df.add_quantile(['hit_rate','num_stops'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state,hit_ratequantiles,num_stopsquantiles
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ,high,low
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ,high,low
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ,high,low
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ,high,low
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ,high,low


In [14]:
labeled_df.get_vars_per_type('categorical')

['state',
 'driver_gender',
 'driver_race',
 'decriminalization',
 'medical',
 'recreational',
 'driver_gender_driver_race',
 'driver_gender_state',
 'driver_race_state',
 'driver_gender_driver_race_state',
 'hit_ratequantiles',
 'num_stopsquantiles']

In [15]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[independent, splitby]",False,NaN
year,int64,ordinal,[independent],False,NaN
driver_gender,object,categorical,"[[independent, dependent], splitby]",False,NaN
driver_race,object,categorical,"[independent, splitby]",False,NaN
decriminalization,int64,categorical,[splitby],False,NaN
medical,int64,categorical,[splitby],False,NaN
recreational,int64,categorical,[splitby],False,NaN
no_reforms,int64,False,[splitby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [16]:
corrobj = wg.All_Pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

[('year', 'search_conducted_rate'),
 ('year', 'contraband_found_rate'),
 ('year', 'hit_rate'),
 ('year', 'num_stops')]

In [17]:
rankobj = wg.Mean_Rank_Trend()
linreg_obj = wg.All_Linear_Trend()

# Computing Trends on a LabeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [18]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()



 df completed
 Index(['independent', 'dependent', 'agg_trend', 'agg_trend_strength',
       'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independe

,dependent,group_feat,independent,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type
0,search_conducted_rate,state,year,AZ,-0.247018,0.247018,pearson_corr,-0.03903,0.03903,aggregate-subgroup
1,search_conducted_rate,state,year,CO,-0.414566,0.414566,pearson_corr,-0.03903,0.03903,aggregate-subgroup
2,search_conducted_rate,state,year,CT,0.118238,0.118238,pearson_corr,-0.03903,0.03903,aggregate-subgroup
3,search_conducted_rate,state,year,IL,-0.199765,0.199765,pearson_corr,-0.03903,0.03903,aggregate-subgroup
4,search_conducted_rate,state,year,MA,-0.603026,0.603026,pearson_corr,-0.03903,0.03903,aggregate-subgroup


Now we can use a list of objects and apply multiple trends

In [19]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup_trend,subgroup_trend_strength,trend_type
103,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.388100,aggregate-subgroup,search_conducted_rate,recreational,state,0,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, RI, WA, I...",0.345200,rank_trend
147,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900,aggregate-subgroup,num_stops,medical,state,1,"[RI, VT, CT, MD, IL, CO, AZ, MA, WA]",0.340200,rank_trend
80,0.316196,0.316196,aggregate-subgroup,hit_rate,recreational,year,0,0.377984,0.377984,pearson_corr
19,-0.0390297,0.039030,aggregate-subgroup,search_conducted_rate,driver_race,year,White,-0.115382,0.115382,pearson_corr
97,-0.043965,0.043965,aggregate-subgroup,num_stops,driver_gender,year,F,-0.0362678,0.036268,pearson_corr
154,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,state,year,CT,-5.2563,0.000831,lin_reg
108,-0.043965,0.043965,aggregate-subgroup,num_stops,recreational,year,0,-0.040337,0.040337,pearson_corr
6,"[Asian, White, Other, Black, Hispanic]",0.277200,aggregate-subgroup,search_conducted_rate,state,driver_race,NC,"[Asian, Other, White, Black, Hispanic]",0.276500,rank_trend
13,"[Asian, White, Other, Black, Hispanic]",0.277200,aggregate-subgroup,search_conducted_rate,driver_gender,driver_race,F,"[Asian, White, Black, Hispanic, Other]",0.213900,rank_trend
33,0.0411292,0.041129,aggregate-subgroup,contraband_found_rate,state,year,MD,0.150174,0.150174,pearson_corr


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [20]:
pd.unique(labeled_df.result_df['trend_type'])

array(['pearson_corr', 'rank_trend', 'lin_reg'], dtype=object)

In [21]:
pd.unique(labeled_df.result_df['comparison_type'])

array(['aggregate-subgroup'], dtype=object)

We can also add trends that are structured for pairwise comparisons

In [22]:
labeled_df.get_pairwise_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)



 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object')


 df completed
 Index(['independent', 'dependent', 'subgroup_trend', 'subgroup_trend_strength',
       'subgroup', 'group_feat', 'trend_type'],
      dtype='object'

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type
7905,NaN,NaN,pairwise,search_conducted_rate,recreational,year,0,1,2.20781,4.84031,0.004293,0.002812,lin_reg
2621,NaN,NaN,pairwise,hit_rate,state,driver_race,CT,VT,"[Other, Black, Hispanic, White, Asian]","[Hispanic, Other, Asian, Black, White]",0.255700,0.144700,rank_trend
5195,NaN,NaN,pairwise,search_conducted_rate,state,year,TX,VT,3.76278,-1.55738,0.014346,0.002015,lin_reg
2466,NaN,NaN,pairwise,hit_rate,state,driver_race,TX,WA,"[Hispanic, Asian, White, Black, Other]","[Asian, Hispanic, Black, Other, White]",0.325500,0.349200,rank_trend
59,NaN,NaN,pairwise,search_conducted_rate,state,year,MA,RI,-0.603026,-0.290574,0.603026,0.290574,pearson_corr
65,"[Asian, Hispanic, Other, Black, White]",0.0023,aggregate-subgroup,hit_rate,recreational,driver_race,0,NaN,"[Hispanic, Asian, Other, Black, White]",NaN,0.008500,NaN,rank_trend
7953,NaN,NaN,pairwise,num_stops,recreational,year,0,1,-1465.04,-10583.1,0.040337,0.146187,lin_reg
3696,NaN,NaN,pairwise,search_conducted_rate,driver_race,state,Asian,Other,"[CO, CT, NC, MD, VT, TX, WI, MA, WA, IL, RI, AZ]","[CO, NC, MA, CT, MD, SC, RI, TX, IL, WI, AZ, V...",0.510900,0.621800,rank_trend
30,"[Asian, White, Other, Hispanic, Black]",0.0716,aggregate-subgroup,contraband_found_rate,state,driver_race,RI,NaN,"[Asian, White, Hispanic, Other, Black]",NaN,0.333400,NaN,rank_trend
7361,NaN,NaN,pairwise,num_stops,driver_gender,year,F,M,-793.986,-2276.91,0.036268,0.051590,lin_reg


In [23]:
pd.unique(labeled_df.result_df['comparison_type'])

array(['aggregate-subgroup', 'pairwise'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [24]:
labeled_df.trend_list

In [25]:
labeled_df.result_df.head()

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type
0,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,state,year,AZ,NaN,-0.247018,NaN,0.247018,NaN,pearson_corr
1,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,state,year,CO,NaN,-0.414566,NaN,0.414566,NaN,pearson_corr
2,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,state,year,CT,NaN,0.118238,NaN,0.118238,NaN,pearson_corr
3,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,state,year,IL,NaN,-0.199765,NaN,0.199765,NaN,pearson_corr
4,-0.0390297,0.03903,aggregate-subgroup,search_conducted_rate,state,year,MA,NaN,-0.603026,NaN,0.603026,NaN,pearson_corr


In [26]:
# a = ['a','b','c','d']
# b = ['a','b','d','e']
a = ['F', 'M']
b = ['F', 'M']
for i, d in enumerate(set(a+b)):
    print (i,':',d)

0 : F
1 : M


In [27]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance(row_wise=True) #('subgroup_trend','subgroup_trend2')

labeled_df.result_df.sample(10)

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
1303,NaN,NaN,pairwise,search_conducted_rate,state,driver_race,MA,MD,"[Other, Asian, White, Black, Hispanic]","[Asian, Other, White, Black, Hispanic]",0.304800,0.206200,rank_trend,0.700000
3577,NaN,NaN,pairwise,search_conducted_rate,driver_gender,driver_race,F,M,"[Asian, White, Black, Hispanic, Other]","[Asian, White, Other, Hispanic, Black]",0.213900,0.372400,rank_trend,0.100000
4737,NaN,NaN,pairwise,search_conducted_rate,state,year,MD,RI,6.78788,2.79571,0.016261,0.015540,lin_reg,0.125567
2711,NaN,NaN,pairwise,hit_rate,state,driver_race,NC,TX,"[Hispanic, Asian, Black, White, Other]","[Hispanic, Asian, White, Black, Other]",0.199900,0.325500,rank_trend,0.300000
186,NaN,NaN,pairwise,contraband_found_rate,state,year,CO,MA,-0.306415,-0.541672,0.306415,0.541672,pearson_corr,0.000000
81,0.316196,0.316196,aggregate-subgroup,hit_rate,recreational,year,1,NaN,-0.0998748,NaN,0.099875,NaN,pearson_corr,1.000000
7674,NaN,NaN,pairwise,num_stops,driver_race,year,Asian,Black,259.628,-2262.72,0.104919,0.169689,lin_reg,0.997267
4979,NaN,NaN,pairwise,search_conducted_rate,state,year,CO,CT,1.61212,-5.2563,0.002736,0.000831,lin_reg,0.526854
1930,NaN,NaN,pairwise,contraband_found_rate,state,driver_race,CT,SC,"[Other, Asian, White, Hispanic, Black]","[Other, White, Black, Hispanic]",0.471600,0.142200,rank_trend,0.400000
4113,NaN,NaN,pairwise,search_conducted_rate,decriminalization,driver_race,0,1,"[Asian, White, Black, Other, Hispanic]","[Other, Asian, White, Black, Hispanic]",0.320400,0.151500,rank_trend,0.500000


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [28]:
labeled_df.trend_list[0].trend_precompute

{'pearson_corr_agg_trend':                            year  contraband_found_rate  search_conducted_rate  \
 year                   1.000000               0.041129              -0.039030   
 contraband_found_rate  0.041129               1.000000               0.807915   
 search_conducted_rate -0.039030               0.807915               1.000000   
 num_stops             -0.043965              -0.073456              -0.065594   
 hit_rate               0.316196               0.275397              -0.094273   
 
                        num_stops  hit_rate  
 year                   -0.043965  0.316196  
 contraband_found_rate  -0.073456  0.275397  
 search_conducted_rate  -0.065594 -0.094273  
 num_stops               1.000000 -0.042102  
 hit_rate               -0.042102  1.000000  ,
 'pearson_corr_subgroup_trend':                                       year  contraband_found_rate  \
 no_reforms                                                          
 0          year                

# Saving with trends

In [29]:
labeled_df.save_all('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI_all')

True

In [30]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI_all
ls

df.csv
meta.csv
result_df.csv
trends.json


In [31]:
labeled_df = wg.LabeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI_all')

/home/smb/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/smb/anaconda3/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['trend_type'] = tt


In [32]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[independent, splitby]",False,NaN
year,int64,ordinal,[independent],False,NaN
driver_gender,object,categorical,"[independent, dependent, splitby]",False,NaN
driver_race,object,categorical,"[independent, splitby]",False,NaN
decriminalization,int64,categorical,[splitby],False,NaN
medical,int64,categorical,[splitby],False,NaN
recreational,int64,categorical,[splitby],False,NaN
no_reforms,int64,False,[splitby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [33]:
labeled_df.trend_list[0].trend_precompute

{'pearson_corr_agg_trend':        year  contraband_found_rate  search_conducted_rate  num_stops  hit_rate
 0  1.000000               0.041129              -0.039030  -0.043965  0.316196
 1  0.041129               1.000000               0.807915  -0.073456  0.275397
 2 -0.039030               0.807915               1.000000  -0.065594 -0.094273
 3 -0.043965              -0.073456              -0.065594   1.000000 -0.042102
 4  0.316196               0.275397              -0.094273  -0.042102  1.000000,
 'pearson_corr_subgroup_trend':        year  contraband_found_rate  search_conducted_rate  num_stops  hit_rate
 0  1.000000               0.118325               0.080876  -0.069907  0.315970
 1  0.118325               1.000000               0.713713  -0.079631  0.392352
 2  0.080876               0.713713               1.000000  -0.050946 -0.099515
 3 -0.069907              -0.079631              -0.050946   1.000000 -0.055962
 4  0.315970               0.392352              -0.099515  -0

# Filtering

In [34]:
help(labeled_df.get_trend_rows)

Help on method get_trend_rows in module wiggum.ranking_processing:

get_trend_rows(independent=None, dependent=None, group_feat=None, subgroup=None, subgroup2=None, trend_type=None, comparison_type=None, inplace=False) method of wiggum.labeled_dataframe.LabeledDataFrame instance
    return a row of result_df based on the specified values. returned rows
    meet provided criteria for all columns (and operator) and any one of the listed
    values for each column (or operator)
    
    Parameters
    -----------
    indep : str, list, or  {None}
        trend variable name or None to include all
    dependent : str, list, or  {None}
        trend variable name or None to include all
    group_feat : str, list, or  {None}
        groupoby variable name or None to include all
    subgroup : str, list, or  {None}
        value of groupby_feat or  None to include all
    subgroup2 : str, list or {None}
        value of groupby_feat or  None to include all
    trend_type: str, list or {None}


So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [36]:
labeled_df.get_trend_rows(independent='year',subgroup=['Black','Hispanic'])

72  total rows meet the criteria


,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
280,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_race,year,Black,NaN,1.24021,NaN,0.001744,NaN,lin_reg,0.132212
281,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_race,year,Hispanic,NaN,5.34096,NaN,0.017419,NaN,lin_reg,0.181954
308,-1560.86,0.043965,aggregate-subgroup,num_stops,driver_race,year,Black,NaN,-2262.72,NaN,0.169689,NaN,lin_reg,0.000127
309,-1560.86,0.043965,aggregate-subgroup,num_stops,driver_race,year,Hispanic,NaN,422.667,NaN,0.014732,NaN,lin_reg,0.998086
2900,NaN,NaN,pairwise,search_conducted_rate,driver_race,year,Black,Hispanic,1.24021,5.34096,0.001744,0.017419,lin_reg,0.314165
2901,NaN,NaN,pairwise,search_conducted_rate,driver_race,year,Black,Other,1.24021,-7.23536,0.001744,0.032087,lin_reg,0.480570
2902,NaN,NaN,pairwise,search_conducted_rate,driver_race,year,Black,White,1.24021,1.9543,0.001744,0.003678,lin_reg,0.130903
2903,NaN,NaN,pairwise,search_conducted_rate,driver_race,year,Black,Hispanic,1.24021,5.34096,0.001744,0.017419,lin_reg,0.314165
2904,NaN,NaN,pairwise,search_conducted_rate,driver_race,year,Black,Other,1.24021,-7.23536,0.001744,0.032087,lin_reg,0.480570
2905,NaN,NaN,pairwise,search_conducted_rate,driver_race,year,Black,White,1.24021,1.9543,0.001744,0.003678,lin_reg,0.130903


In [37]:
labeled_df.get_trend_rows(group_feat = 'driver_race',trend_type ='lin_reg' )

330  total rows meet the criteria


,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
279,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_race,year,Asian,NaN,1.24187,NaN,0.003783,NaN,lin_reg,0.131795
280,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_race,year,Black,NaN,1.24021,NaN,0.001744,NaN,lin_reg,0.132212
281,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_race,year,Hispanic,NaN,5.34096,NaN,0.017419,NaN,lin_reg,0.181954
282,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_race,year,Other,NaN,-7.23536,NaN,0.032087,NaN,lin_reg,0.612781
283,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_race,year,White,NaN,1.9543,NaN,0.003678,NaN,lin_reg,0.001309
307,-1560.86,0.043965,aggregate-subgroup,num_stops,driver_race,year,Asian,NaN,259.628,NaN,0.104919,NaN,lin_reg,0.997140
308,-1560.86,0.043965,aggregate-subgroup,num_stops,driver_race,year,Black,NaN,-2262.72,NaN,0.169689,NaN,lin_reg,0.000127
309,-1560.86,0.043965,aggregate-subgroup,num_stops,driver_race,year,Hispanic,NaN,422.667,NaN,0.014732,NaN,lin_reg,0.998086
310,-1560.86,0.043965,aggregate-subgroup,num_stops,driver_race,year,Other,NaN,91.5127,NaN,0.075180,NaN,lin_reg,0.992636
311,-1560.86,0.043965,aggregate-subgroup,num_stops,driver_race,year,White,NaN,-5591.73,NaN,0.097383,NaN,lin_reg,0.000294


In [38]:
labeled_df.result_df.columns

Index(['agg_trend', 'agg_trend_strength', 'comparison_type', 'dependent',
       'group_feat', 'independent', 'subgroup', 'subgroup2', 'subgroup_trend',
       'subgroup_trend2', 'subgroup_trend_strength',
       'subgroup_trend_strength2', 'trend_type', 'distance'],
      dtype='object')

We can also filter based on SP detections with `

In [39]:
labeled_df.get_SP_rows(thresh=.2)

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2
264,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,state,year,AZ,NaN,0.0220027,NaN,0.001366,NaN,lin_reg,0.686210,True
266,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,state,year,CT,NaN,-5.2563,NaN,0.000831,NaN,lin_reg,0.580529,True
269,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,state,year,MD,NaN,6.78788,NaN,0.016261,NaN,lin_reg,0.206667,True
270,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,state,year,NC,NaN,0.652098,NaN,0.007841,NaN,lin_reg,0.332345,True
272,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,state,year,SC,NaN,-0.220005,NaN,0.000837,NaN,lin_reg,0.838078,True
274,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,state,year,VT,NaN,-1.55738,NaN,0.002015,NaN,lin_reg,0.336829,True
276,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,state,year,WI,NaN,0.60121,NaN,0.000842,NaN,lin_reg,0.355607,True
277,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_gender,year,F,NaN,0.49675,NaN,0.001322,NaN,lin_reg,0.406705,True
282,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,driver_race,year,Other,NaN,-7.23536,NaN,0.032087,NaN,lin_reg,0.612781,True
284,1.96425,0.004473,aggregate-subgroup,search_conducted_rate,decriminalization,year,0,NaN,0.740524,NaN,0.005495,NaN,lin_reg,0.294428,True


## Detection

Detection via `get_SP_rows` happens in two steps:
1. label the rows
2. filter by that column to return

Labeling the rows can happen in a number of ways too, the detection accepts a number of forms of input, custom detections can be built in many ways

In [40]:
help(labeled_df.label_SP_rows)

Help on method label_SP_rows in module wiggum.ranking_processing:

label_SP_rows(filter_thresh=None) method of wiggum.labeled_dataframe.LabeledDataFrame instance
    update the result_df with an additional colulmn indicateing rows with SP
    (or SP-like) as defined by sp_type
    
    Parameters
    -----------
    
    self : LabeledDataFrame
        must have values in result_df
    filter_thresh : dict or string
        dictionary of column label, threshold pairs or string name of a
        prespecified dictionary if dict, must include 'name' field (which
        will be used as the column name for storing the detections)



when filter_thresh is a dictionary, the filtering happens by taking the intersection of each row by the treshold prvided.  Some defaults are also built in accessible by string.

In [41]:
wg.trend_quality_sp

{'agg_trend_strength': 0.15,
 'distance': 0.2,
 'name': 'default_qual_sp',
 'subgroup_trend_strength': 0.15}

Which can be applied with:

In [42]:
labeled_df.get_SP_rows('default_qual_sp')

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp
64,0.316196,0.316196,aggregate-subgroup,hit_rate,state,year,SC,NaN,-0.255096,NaN,0.255096,NaN,pearson_corr,1.0000,True,True
67,0.316196,0.316196,aggregate-subgroup,hit_rate,state,year,WA,NaN,-0.68403,NaN,0.684030,NaN,pearson_corr,1.0000,True,True
114,['Asian' 'White' 'Other' 'Black' 'Hispanic'],0.277200,aggregate-subgroup,search_conducted_rate,state,driver_race,CT,NaN,['Asian' 'Other' 'White' 'Hispanic' 'Black'],nan,0.443000,NaN,rank_trend,0.6000,True,True
116,['Asian' 'White' 'Other' 'Black' 'Hispanic'],0.277200,aggregate-subgroup,search_conducted_rate,state,driver_race,MA,NaN,['Other' 'Asian' 'White' 'Black' 'Hispanic'],nan,0.304800,NaN,rank_trend,0.6000,True,True
117,['Asian' 'White' 'Other' 'Black' 'Hispanic'],0.277200,aggregate-subgroup,search_conducted_rate,state,driver_race,MD,NaN,['Asian' 'Other' 'White' 'Black' 'Hispanic'],nan,0.206200,NaN,rank_trend,0.5000,True,True
118,['Asian' 'White' 'Other' 'Black' 'Hispanic'],0.277200,aggregate-subgroup,search_conducted_rate,state,driver_race,NC,NaN,['Asian' 'Other' 'White' 'Black' 'Hispanic'],nan,0.276500,NaN,rank_trend,0.5000,True,True
119,['Asian' 'White' 'Other' 'Black' 'Hispanic'],0.277200,aggregate-subgroup,search_conducted_rate,state,driver_race,RI,NaN,['Other' 'Asian' 'White' 'Hispanic' 'Black'],nan,0.477600,NaN,rank_trend,0.7000,True,True
120,['Asian' 'White' 'Other' 'Black' 'Hispanic'],0.277200,aggregate-subgroup,search_conducted_rate,state,driver_race,SC,NaN,['Other' 'White' 'Black' 'Hispanic'],nan,0.414300,NaN,rank_trend,0.5000,True,True
123,['Asian' 'White' 'Other' 'Black' 'Hispanic'],0.277200,aggregate-subgroup,search_conducted_rate,state,driver_race,WA,NaN,['Asian' 'White' 'Hispanic' 'Black' 'Other'],nan,0.528600,NaN,rank_trend,0.3000,True,True
128,['Asian' 'White' 'Other' 'Black' 'Hispanic'],0.277200,aggregate-subgroup,search_conducted_rate,decriminalization,driver_race,1,NaN,['Other' 'Asian' 'White' 'Black' 'Hispanic'],nan,0.151500,NaN,rank_trend,0.6000,True,True


In [43]:
wg.DEFAULT_SP_DEF

{'comparison_type': 'aggregate', 'distance': 0.0, 'name': 'SP'}

Which can be applied with:

In [44]:
labeled_df.get_SP_rows('SP')

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP


We can also define our own detection filters, using any available column

In [45]:
lin_only_qual = {'name':'lin_only_qual_sp','distance':.2, 'agg_trend_strength':.05,
                'subgroup_trend_strength':.15,'trend_type':'lin_reg'}
labeled_df.get_SP_rows(lin_only_qual,replace=True) 

,agg_trend,agg_trend_strength,comparison_type,dependent,group_feat,independent,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP,lin_only_qual_sp


# Ranking

In [ ]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

In [ ]:
labeled_df.add_view_score('SP_thresh0.2',agg_type='sum',colored=False).head(10)

In [ ]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh0.2','SP_thresh0.2').head()